In [ ]:
import configparser

from azure.ai.ml import Input, load_component, MLClient
from azure.ai.ml.entities import AmlCompute
from azure.ai.ml.dsl import pipeline
from azure.identity import DefaultAzureCredential

In [ ]:
config = configparser.ConfigParser()
config.read('config.ini')
subscription_id = config.get('Azure', 'subscription_id')
resource_group = config.get('Azure', 'resource_group')
workspace_name = config.get('Azure', 'workspace')
datastore_name = config.get('Azure', 'datastore_name')

In [ ]:
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

In [ ]:
cpu_compute_target = "cpu-cluster"

try:
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new cpu compute target...")
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        type="amlcompute",
        size="STANDARD_DS3_V2",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=180,
        tier="Dedicated",
    )
    print(
        f"AMLCompute with name {cpu_cluster.name} will be created, with compute size {cpu_cluster.size}"
    )
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)

In [ ]:
component_names = ['description', 'clean', 'correlation', 'split', 'logistic_regression_train', 'decission_trees_train', 'score', 'evaluate']
components = {}
for name in component_names:
    component = load_component(source=f"./components/{name}_component/{name}.yml")
    components[name] = ml_client.create_or_update(component)


In [ ]:
@pipeline(
    default_compute=cpu_compute_target
)
def water_potability_logistic_regression(pipeline_input_data):
    correlation_node = components['correlation'](
        data=pipeline_input_data
    )
    clean_node = components['clean'](
        data=pipeline_input_data
    )
    split_node = components['split'](
        split_data=clean_node.outputs.clean_data_output
    )
    logistic_regression_train_node = components['logistic_regression_train'](
        train_data=split_node.outputs.train_output,
        objective='Potability'
    )
    score_node = components['score'](
        model=logistic_regression_train_node.outputs.model_output,
        test_data=split_node.outputs.test_output,
    )
    evaluate_node = components['evaluate'](
        test_data=split_node.outputs.test_output,
        target_names='Not Potable,Potable',
        predict_data=score_node.outputs.predict_output
    )
    return {
        'pairplot': correlation_node.outputs.pairplot_image_output,
        'model': logistic_regression_train_node.outputs.model_output,
        'report': evaluate_node.outputs.report_output
    }

In [ ]:
data_asset = ml_client.data.get(name="water-potability", version='1')
water_potability = Input(type="uri_file", path=data_asset.path)
pipeline = water_potability_logistic_regression(pipeline_input_data=water_potability)

In [ ]:
pipeline_job = ml_client.jobs.create_or_update(
    pipeline,
    experiment_name='pipeline-exp',
)
ml_client.jobs.stream(pipeline_job.name)

In [ ]:
output = ml_client.jobs.download(name=pipeline_job.name, download_path='./pipeline_output', all=True)